## 集成学习

集成学习(ensemble learning)是现在非常火爆的机器学习方法。   
**它本身不是一个单独的机器学习算法，而是通过构建并结合多个机器学习器来完成学习任务**。
也就是我们常说的“博采众长”。   

**集成学习可以用于分类问题集成，回归问题集成，特征选取集成，异常点检测集成等等**，可以说所有的机器学习领域都可以看到集成学习的身影。

### 集成学习概述
从下图，我们可以对集成学习的思想做一个概括。
对于训练集数据，我们通过训练若干个个体学习器，通过一定的结合策略，就可以最终形成一个强学习器，以达到博采众长的目的。

![jupyter](./images/640.png)
也就是说，集成学习有两个主要的问题需要解决：
- 第一是如何得到若干个个体学习器，
- 第二是如何选择一种结合策略，将这些个体学习器集合成一个强学习器。

### EL之个体学习器

如上所述，集成学习的第一个问题就是如何得到若干个个体学习器。这里我们有两种选择。

第一种就是**所有的个体学习器都是一个种类的，或者说是同质的。比如都是决策树个体学习器，或者都是神经网络个体学习器**。

第二种就是**所有的个体学习器不全是一个种类的，或者说是异质的。比如我们有一个分类问题，对训练集采用支持向量机个体学习器，逻辑回归个体学习器和朴素贝叶斯个体学习器来学习，再通过某种结合策略来确定最终的分类强学习器。**

目前来说，**同质个体学习器的应用是最广泛的，一般我们常说的集成学习的方法都是指的同质个体学习器**。而同质个体学习器使用最多的模型是CART决策树和神经网络。

同质个体学习器按照个体学习器之间**是否存在依赖关系**可以分为两类：
- 第一个是个体学习器之间**存在强依赖关系**，一系列个体学习器基本都需要**串行生成**，代表算法是**boosting系列算法**，
- 第二个是个体学习器之间**不存在强依赖关系**，一系列个体学习器可以**并行生成**，代表算法是**bagging和随机森林（Random Forest）系列算法**。

### boosting
Boosting的算法原理我们可以用一张图做一个概括如下：
![jupyter](./images/641.jpeg)

从图中可以看出，Boosting算法的工作机制是：
- 首先从训练集用初始权重训练出一个弱学习器1，根据弱学习的学习误差率表现来更新训练样本的权重，使得之前弱学习器1学习误差率高的训练样本点的权重变高，使得这些误差率高的点在后面的弱学习器2中得到更多的重视。
- 然后基于调整权重后的训练集来训练弱学习器2，如此重复进行，直到弱学习器数达到事先指定的数目T，最终将这T个弱学习器通过集合策略进行整合，得到最终的强学习器。

Boosting系列算法里最著名算法主要有AdaBoost算法和提升树(boosting tree)系列算法。提升树系列算法里面应用最广泛的是梯度提升树(Gradient Boosting Tree)。

### Bagging

Bagging的算法原理和 boosting不同，它的弱学习器之间没有依赖关系，可以并行生成，我们可以用一张图做一个概括如下：
![jupyter](./images/642.jpeg)
从上图可以看出，bagging的个体弱学习器的训练集是通过**随机采样**得到的。
**通过T次的随机采样，我们就可以得到T个采样集，对于这T个采样集，我们可以分别独立的训练出T个弱学习器，再对这T个弱学习器通过集合策略来得到最终的强学习器。**


对于这里的随机采样有必要做进一步的介绍，这里一般采用的是**自助采样法（Bootstap sampling）,即对于m个样本的原始训练集，我们每次先随机采集一个样本放入采样集，接着把该样本放回，也就是说下次采样时该样本仍有可能被采集到，这样采集m次，最终可以得到m个样本的采样集**，由于是随机采样，这样每次的采样集是和原始训练集不同的，和其他采样集也是不同的，这样得到多个不同的弱学习器。


随机森林是bagging的一个特化进阶版，所谓的特化是因为**随机森林的弱学习器都是决策树**。
所谓的进阶是随机森林在bagging的**样本随机采样**基础上，又加上了**特征的随机选择**，其基本思想没有脱离bagging的范畴。

### 结合策略

- **平均法**
    对于数值类的**回归预测**问题，通常使用的结合策略是平均法，也就是说，对于若干和弱学习器的输出进行平均得到最终的预测输出。最简单的平均是算术平均，也就是说最终预测是
    如果每个个体学习器有一个权重w，则最终预测是

    其中wi是个体学习器hi的权重，通常有

    ![jupyter](./images/643.jpeg)


- **投票法**
    对于**分类问题的预测**，我们通常使用的是投票法。
    假设我们的预测类别是{c1,c2,...cK},对于任意一个预测样本x，我们的T个弱学习器的预测结果分别是(h1(x),h2(x)...hT(x))。

    最简单的投票法是相对多数投票法，也就是我们常说的少数服从多数，也就是T个弱学习器的对样本x的预测结果中，数量最多的类别ci为最终的分类类别。如果不止一个类别获得最高票，则随机选择一个做最终类别。
    稍微复杂的投票法是绝对多数投票法，也就是我们常说的要票过半数。在相对多数投票法的基础上，不光要求获得最高票，还要求票过半数。否则会拒绝预测。
    更加复杂的是加权投票法，和加权平均法一样，每个弱学习器的分类票数要乘以一个权重，最终将各个类别的加权票数求和，最大的值对应的类别为最终类别。
    
- **学习法(stacking)**
    上述方法都是对弱学习器的结果做平均或者投票，相对比较简单，但是可能学习误差较大，于是就有了学习法这种方法，对于学习法，代表方法是stacking，当使用stacking的结合策略时， 我们不是对弱学习器的结果做简单的逻辑处理，而是再加上一层学习器，也就是说，我们将训练集弱学习器的学习结果作为输入，将训练集的输出作为输出，重新训练一个学习器来得到最终结果。
    在这种情况下，我们将弱学习器称为初级学习器，将用于结合的学习器称为次级学习器。对于测试集，我们首先用初级学习器预测一次，得到次级学习器的输入样本，再用次级学习器预测一次，得到最终的预测结果。